# Utilizing KNN Classification to Predict Whether or not a Patient with the Specified Parameters will have Heart Disease
DSCI 100: Group 22 Project Proposal

### Introduction

Cardiovascular disease, which encompasses all kinds of heart disease, is the leading cause of death globally. Accurate diagnosis in the early stage is crucial to effectively treat and prevent possible heart disease. The question we ask is: Using a patient’s age, resting blood pressure, serum cholesterol level, and maximum heart rate achieved, how well can we predict the presence of heart disease using a K-nearest neighbors classifier? Our dataset is the Heart Failure Prediction dataset from Kaggle, which is a combination of five different datasets from the UCI Machine Learning Repository heart disease index, using 11 common features and one common label. These variables are *Age* for age in years, *Sex* for gender, *ChestPainType* for the type of angina, *RestingBP* for resting blood pressure, *Cholesterol* for serum cholesterol levels in blood, FastingBS for fasting blood sugar, *RestingECG* for resting electrocardiogram results, *MaxHR* for maximum heart rate achieved, *ExerciseAngina* for the angina is exercise-induced, *OldPeak* for ST depression induced by exercise relative to rest, *ST_Slope* for the slope of the peak exercise ST segment, and *HeartDisease* for the presence of heart disease.

### Methods and Results

The dataset contains general information for patients, as well as information about or related to the heart. There are 12 variables and 918 observations. Our main categorical label for this dataset, the presence of heart disease, is the *HeartDisease* column. *HeartDisease* can either be 0, meaning no heart disease, or 1, indicating the presence of heart disease.


In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

-- Attaching packages --------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.4     v dplyr   1.0.7
v tidyr   1.1.3     v stringr 1.4.0
v readr   2.0.1     v forcats 0.5.1

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Warning message:
"package 'tidymodels' was built under R version 4.1.2"


ERROR: Error: package or namespace load failed for 'tidymodels' in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 namespace 'tidyr' 1.1.3 is already loaded, but >= 1.1.4 is required


We set the seed to make the code reproducible.

In [ ]:
set.seed(6)

Since downloading datasets from Kaggle requires an account, we have uploaded the dataset to a GitHub repository and have downloaded the csv from the repository instead.

In [ ]:
url <- "https://raw.githubusercontent.com/Stevan-Zhuang/DSCI-100-group-project-22/main/heart.csv"
download.file(url, "heart-disease.csv")
heart <- read_csv("heart-disease.csv")
heart

The dataset contains general information for patients, as well as information related to the heart. There are 12 variables and 918 observations. The main categorical label that we want to predict is the *HeartDisease* column. *HeartDisease* can either be 0, meaning no heart disease, or 1, indicating the presence of heart disease.

We see in the dataframe *HeartDisease* is of type double. *HeartDisease* is a categorical variable, not a numerical one, so we convert it to factor.

In [ ]:
heart <- heart %>%
    mutate(HeartDisease = as_factor(HeartDisease))
heart

Since we are using a KNN classifier, we can only use the numerical features. We select only the columns with numerical data, which are *Age*, *RestingBP*, *Cholesterol* and *MaxHR*, along with the target variable.

In [ ]:
heart <- heart %>% 
    select(Age, RestingBP, Cholesterol, MaxHR, HeartDisease)
heart

We will need separate training and testing datasets to make sure the model is never trained on observations it is tested on. We do this by splitting our original dataset, with 80% of the observations going to the training dataset and 20% of the observations going to the testing dataset.

In [ ]:
heart_split <- heart %>% 
initial_split(fold = 4/5, strata = HeartDisease)
heart_train <- training(heart_split)
heart_test <- testing(heart_split)
heart_train 
heart_test

It would be helpful to see the distribution of the predicted variable to check if there are an equal amount of both possible values. If the distribution is unevenly split, we should take it into account when reviewing the metrics for our model.

In [ ]:
heart_count <- heart_train %>%
    group_by(HeartDisease) %>%
    summarize(count = n())

heart_count
heart_0_count <- heart_count %>% slice(1) %>% pull()
heart_1_count <- heart_count %>% slice(2) %>% pull()

heart_1_count / (heart_0_count + heart_1_count)

There are 307 observations of patients with heart disease, and 381 observations of patients without, with patients having heart disease accounting for roughly 55% of all patients. This means there are more patients with heart disease than without in our dataset.

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)

heart_train %>%
    pivot_longer(Age:MaxHR, names_to = "column", values_to = "value") %>%
    ggplot(aes(x = value)) +
    geom_histogram(aes(fill = HeartDisease), bins = 90) +
    facet_grid(column ~ .) +
    labs(x = "Values", y = "Number of Values",
         title = "Distribution of Each Predictor Variable",
         fill = "Has Heart Disease? (0: No, 1: Yes)",
         subtitle = "Figure 1") +
    theme(text = element_text(size = 20))

The plots above show the distribution of patients with/without heart disease for the predictor variables used. The distribution for *Cholesterol* is noticeably different from the other distributions and seems to contain a large amount of zero values. Since having a concentration of 0 mm/dl of cholesterol is unrealistic, it is likely a missing value and we should consider not using the *Cholesterol* variable or removing observations with *Cholesterol* values of 0.

In [ ]:
heart_train %>%
    filter(Cholesterol != 0) %>%
    pivot_longer(Age:MaxHR, names_to = "column", values_to = "value") %>%
    ggplot(aes(x = value)) +
    geom_histogram(aes(fill = HeartDisease), bins = 90) +
    facet_grid(column ~ .) +
    labs(x = "Values", y = "Number of Values",
         title = "Distribution of Each Predictor Variable where Cholesterol is Non-Zero",
         fill = "Has Heart Disease? (0: No, 1: Yes)",
         subtitle = "Figure 2") +
    theme(text = element_text(size = 20))

Even with the zero values removed, the distribution for *Cholesterol* is noticeably more spread out than all of the other features. This tells us that scaling and centering the data will likely help the KNN classifier do better.

To create and train our KNN classifier, we will need to make a recipe, model and workflow. We start by making the recipe, predicting *HeartDisease* using other available variables along with scaling and centering the data.

In [ ]:
heart_recipe <- recipe(HeartDisease ~ ., data = heart_train) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors())
heart_recipe

To get more accurate metrics when training, we will use cross validation with a k value of 5 to split our training dataset into five parts, averaging results while alternating each part as our validation set.

In [ ]:
heart_cv <- heart_train %>%
    vfold_cv(v = 5, strata = HeartDisease)

Since we do not know the optimal value of k for our knn classifier, we should test a range of k values to see what can get the best accuracy without underfitting or overfitting.

In [ ]:
heart_knn <- nearest_neighbor(weight = "rectangular", neighbors = tune()) %>%
    set_engine("kknn") %>%
    set_mode("classification")
heart_knn

Here we test our model on all odd k values from 1 to 50 and graph the results. We only test for odd values because an even value of neighbors can cause draws due to an equal amount of neighbors for each class.

In [ ]:
gridvals = tibble(neighbors = seq(from = 1, to = 50, by = 2))

grid_metrics <- workflow() %>%
    add_recipe(heart_recipe) %>%
    add_model(heart_knn) %>%
    tune_grid(resamples = heart_cv, grid = gridvals) %>%
    collect_metrics() %>%
    filter(.metric == "accuracy")
grid_metrics

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 5)

grid_metrics %>%
    ggplot(aes(neighbors, mean)) +
    geom_point() +
    geom_line() +
    labs(x = "Number of neighbors", y = "Accuracy",
         title = "", subtitle = "Figure 3") +
    scale_x_continuous(breaks = seq(from = 1, to = 50, by = 2)) +
    theme(text = element_text(size = 20))

The value for the number of neighbors that has the highest accuracy is 29, so we select it as our K value.

In [ ]:
k_opt <- grid_metrics %>%
    arrange(desc(mean)) %>%
    select(neighbors) %>%
    slice(1) %>%
    pull()
k_opt

Now we train our model again using the optimal value of K we found earlier on the training dataset.

In [ ]:
heart_knn_tuned <- nearest_neighbor(weight = "rectangular", neighbors = k_opt) %>%
    set_engine("kknn") %>%
    set_mode("classification")
heart_knn

In [ ]:
heart_knn_fit <- workflow() %>%
    add_recipe(heart_recipe) %>%
    add_model(heart_knn_tuned) %>%
    fit(heart_train)
heart_knn_fit

With our model trained, we have it predict on the training data and compare it with the actual labels for each observation to find the average accuracy of the model.

In [ ]:
heart_knn_fit %>%
    predict(heart_train) %>%
    bind_cols(heart_train) %>%
    metrics(truth = HeartDisease, .pred_class) %>%
    filter(.metric == "accuracy") %>%
    select(.estimate) %>%
    slice(1) %>%
    pull()

The accuracy of our model on the training data is roughly 74%, meaning for every incorrect guess the model makes, it makes a bit under three correct guesses. Since we trained our model on the training data itself, we should expect our model to have lower accuracy on the testing data.

We would like to visualize our model predictions, in order to interpret the effect of each predictor variable on the on the final result for the outcome variable. To do this, we plot our data as a scatter plot and colour the points by the presence of heart disease, and fill the background with a grid coloured by the prediction of the model at those points. However, since we have four predictor variables, there are too many dimensions to graph all at once. To get around this problem, we will graph our variables in pairs, and set the values of the other variables as samples from the dataset.

In [ ]:
install.packages("pracma")
library(pracma)

In order to make the visualization above, .

In [ ]:
step = 50

age_min <- heart_train %>% summarize(m = min(Age)) %>% pull()
age_max <- heart_train %>% summarize(m = max(Age)) %>% pull()
restingbp_min <- heart_train %>% summarize(m = min(RestingBP)) %>% pull()
restingbp_max <- heart_train %>% summarize(m = max(RestingBP)) %>% pull()

age_step <- (age_max - age_min) / step
restingbp_step <- (restingbp_max - restingbp_min) / step

age_range = seq(age_min, age_max, age_step)
restingbp_range = seq(restingbp_min, restingbp_max, restingbp_step)

age_restingbp_grid = meshgrid(age_range, restingbp_range)
age_vals = age_restingbp_grid[[1]]
restingbp_vals = age_restingbp_grid[[2]]

age_l_range = seq(age_min - age_step/2, age_max - age_step/2, age_step)
restingbp_l_range = seq(restingbp_min - restingbp_step/2,
                        restingbp_max - restingbp_step/2, restingbp_step)
age_restingbp_l_grid = meshgrid(age_l_range, restingbp_l_range)
age_l_vals = age_restingbp_l_grid[[1]]
restingbp_l_vals = age_restingbp_l_grid[[2]]

age_r_range = seq(age_min + age_step/2, age_max + age_step/2, age_step)
restingbp_r_range = seq(restingbp_min + restingbp_step/2,
                        restingbp_max + restingbp_step/2, restingbp_step)
age_restingbp_r_grid = meshgrid(age_r_range, restingbp_r_range)
age_r_vals = age_restingbp_r_grid[[1]]
restingbp_r_vals = age_restingbp_r_grid[[2]]

cholesterol_mean <- heart_train %>% summarize(m = mean(Cholesterol)) %>% pull()
cholesterol_sd <- heart_train %>% summarize(sd = sd(Cholesterol)) %>% pull()
maxhr_mean <- heart_train %>% summarize(m = mean(MaxHR)) %>% pull()
maxhr_sd <- heart_train %>% summarize(sd = sd(MaxHR)) %>% pull()

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)

grid <- tibble(age_l = c(age_l_vals),
               age_r = c(age_r_vals),
               restingbp_l = c(restingbp_l_vals),
               restingbp_r = c(restingbp_r_vals),
               Age = c(age_vals),
               RestingBP = c(restingbp_vals),
               Cholesterol = rnorm((step + 1)^2, mean = cholesterol_mean, sd = cholesterol_sd),
               MaxHR = rnorm((step + 1)^2, mean = maxhr_mean, sd = maxhr_sd))

grid_predictions <- heart_knn_fit %>%
    predict(grid) %>%
    bind_cols(grid)

ggplot() +
    geom_rect(data = grid_predictions,
              mapping = aes(xmin = age_l, xmax = age_r,
                            ymin = restingbp_l, ymax = restingbp_r,
                            fill = .pred_class),
              alpha = 0.3) +
    geom_point(data = heart_train, mapping = aes(Age, RestingBP, color = HeartDisease)) +
    labs(color = "Patient status",
         fill = "Model prediction") +
    theme(text = element_text(size = 20))

In [ ]:
grid

In [ ]:
cholesterol_min <- heart_train %>% summarize(m = min(Cholesterol)) %>% pull()
cholesterol_max <- heart_train %>% summarize(m = max(Cholesterol)) %>% pull()
maxhr_min <- heart_train %>% summarize(m = min(MaxHR)) %>% pull()
maxhr_max <- heart_train %>% summarize(m = max(MaxHR)) %>% pull()

cholesterol_step <- (cholesterol_max - cholesterol_min) / step
maxhr_step <- (maxhr_max - maxhr_min) / step

age_mean <- heart_train %>% summarize(m = mean(Age)) %>% pull()
age_sd <- heart_train %>% summarize(sd = sd(Age)) %>% pull()
restingbp_mean <- heart_train %>% summarize(m = mean(RestingBP)) %>% pull()
restingbp_sd <- heart_train %>% summarize(sd = sd(RestingBP)) %>% pull()

In [ ]:
heart_knn_fit %>%
    predict(heart_test) %>%
    bind_cols(heart_test) %>%
    metrics(truth = HeartDisease, .pred_class) %>%
    filter(.metric == "accuracy") %>%
    select(.estimate) %>%
    slice(1) %>%
    pull()